In [15]:
import pandas
import urllib
import json
import requests
import os
host = 'https://www.encodeproject.org'
experiments = "https://www.encodeproject.org/experiments/"
sample_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_file_manifest_from_website.txt'
control_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_control_file_manifest_from_website.txt'
combined_sample_and_control_manifest = '/home/bay001/projects/encode/permanent_data/rna_seq_manifest.txt'
out_dir = '/projects/ps-yeolab3/encode/rnaseq/differential_expression/'
out_dir2 = '/oasis/tscc/scratch/bay001/encode/data/'
to_download = '/home/bay001/projects/encode/permanent_data/to_download.sh'
to_download2 = '/home/bay001/projects/encode/permanent_data/to_download2.sh'

### You'll need and manifest file from Eric that has complete imformation on the RBP, Cell Type, CLIP and RNASeq Encode IDs for the experiments. The experiment IDS contain a Accession ID for the whole experiment and then individual ENC IDs for each replicate ( KD and control ). This goes for both CLIP and RNASeq. You'll also need the bam IDs for each replicate

In [16]:
def star_or_tophat(link):
    if os.path.basename(link).find('star') > -1:
        return 'STAR'
    if os.path.basename(link).find('tophat') > -1:
        return 'TOPHAT'
    return 'ambiguous'

In [17]:
X = pandas.read_table('/home/gpratt/Dropbox/encode_integration/20160408_ENCODE_MASTER_ID_LIST_AllDatasets.csv',index_col=0)
X.head(2)

,uID,RBP_gID,CellLine,CLIP_rep1,CLIP_rep2,INPUT,RNASEQ_ControlRep1Bam,RNASEQ_ControlRep2Bam,RNASEQ_ControlENC,RNASEQ_KDRep1Bam,RNASEQ_KDRep2Bam,RNASEQ_ENCODEAccID,selection_method,RBP_ENSG
0,202,PTBP1,HepG2,/projects/ps-yeolab2/encode/analysis/encode_v1...,/projects/ps-yeolab2/encode/analysis/encode_v1...,/projects/ps-yeolab2/encode/analysis/encode_v1...,ENCFF571ARP.bam,ENCFF755GKJ.bam,ENCSR603TCV,ENCFF527FPF.bam,ENCFF155FIK.bam,ENCSR064DXG,polyadenylated mRNA,ENSG00000011304.12
1,203,HNRNPC,HepG2,/projects/ps-yeolab2/encode/analysis/encode_v1...,/projects/ps-yeolab2/encode/analysis/encode_v1...,/projects/ps-yeolab2/encode/analysis/encode_v1...,ENCFF460AAE.bam,ENCFF058QDT.bam,ENCSR305XWT,ENCFF031QZG.bam,ENCFF531AUK.bam,ENCSR052IYH,polyadenylated mRNA,ENSG00000092199.13


In [4]:
testsubset = X[X['CellLine']=='K562']['RNASEQ_ENCODEAccID'].dropna()[:1]
testsubsetcontrol = X[X['CellLine']=='K562']['RNASEQ_ControlENC'].dropna()[:1]
testsubset

13    ENCSR629EWX
Name: RNASEQ_ENCODEAccID, dtype: object

##### Julia found some BAMs that exist on ENCODEPROJECT.ORG but do not show up in my manifest. Are they in Gabe's manifest?
 

In [21]:
# X[(X['CellLine']=='K562') & (X['RBP_gID']=='DKC1')]
X[(X['CellLine']=='HepG2') & (X['RBP_gID']=='LARP4')]


,uID,RBP_gID,CellLine,CLIP_rep1,CLIP_rep2,INPUT,RNASEQ_ControlRep1Bam,RNASEQ_ControlRep2Bam,RNASEQ_ControlENC,RNASEQ_KDRep1Bam,RNASEQ_KDRep2Bam,RNASEQ_ENCODEAccID,selection_method,RBP_ENSG
206,558,LARP4,HepG2,/projects/ps-yeolab2/encode/analysis/encode_v1...,/projects/ps-yeolab2/encode/analysis/encode_v1...,/projects/ps-yeolab2/encode/analysis/encode_v1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000161813.16


### This block scrolls through the Encode webpage JSON code to extract the correct locations / links to the KD bam files. Links to control bam files are generated further below. For now, names of control bam files are stored in a dictionary, with the accession number as key and the bam file name as value

### This should follow the schema from (as of 9-8-2016): https://github.com/ENCODE-DCC/encoded/blob/master/src/encoded/schemas/file.json

In [7]:
samples = list()
links = list()
controls = list()
error = list()
for h in X['RNASEQ_ENCODEAccID'].dropna():
    url = experiments+h+"/?format=json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    if 'code' in data.keys():
        next
    else:
        for i in range(0,len(data['files'])):
            # for key, value in data['files'][i]['md5sum'].iteritems():
            #     print(key)
            
            # print(data['files'][i]['md5sum'])
            # print(data['files'][i]['content_md5sum'])
            # print( data['files'][i]['replicate']['experiment']['target']['name'])
            if ((host+data['files'][i]['href']).endswith('bam') & 
            (data['files'][i]['output_type'] == u'alignments') &
            (data['files'][i]['lab'][u'name'] == 'encode-processing-pipeline')):
                if(list(X[X['RNASEQ_ENCODEAccID']==h]['RNASEQ_ControlENC'])[0] != data['possible_controls'][0]['accession']):
                    error.append("WARNING: CTRL VS AFF CHANGED: {}:{}:{}".format(
                        h,
                        list(X[X['RNASEQ_ENCODEAccID']==h]['RNASEQ_ControlENC'])[0],
                        data['possible_controls'][0]['accession']))
                samples.append({'expt_name':data['files'][i]['replicate']['experiment']['target']['name'],
                                'expt_cell_line':data['files'][i]['replicate'][u'library']['biosample']['biosample_term_name'],
                                'biological_replicate_number':data['files'][i]['replicate']['biological_replicate_number'],
                                'submitted_expt_file_name':data['files'][i]['submitted_file_name'],
                                'expt_file_name':os.path.basename(data['files'][i]['href']),
                                'assembly':data['files'][i]['assembly'],
                                'expt_url':host+data['files'][i]['href'],
                                'RNASEQ_ControlENC':data['possible_controls'][0]['accession'],
                                'RNASEQ_ENCODEAccID':h,
                                'aligner': star_or_tophat(data['files'][i]['submitted_file_name']),
                                'expt_md5sum': data['files'][i]['md5sum']
                               })
                links.append(host+data['files'][i]['href'])
        controls.append(data['possible_controls'][0]['accession'])
samples_df = pandas.DataFrame(samples)
samples_df.to_csv(sample_manifest,
                  sep='\t',
                  index=None)
samples_df

,RNASEQ_ControlENC,RNASEQ_ENCODEAccID,aligner,assembly,biological_replicate_number,expt_cell_line,expt_file_name,expt_md5sum,expt_name,expt_url,submitted_expt_file_name
0,ENCSR603TCV,ENCSR064DXG,TOPHAT,hg19,1,HepG2,ENCFF019ZBE.bam,5a40522cdaea3cef922ca8a2cd2524b4,PTBP1-human,https://www.encodeproject.org/files/ENCFF019ZB...,/long-RNA-seq/runs/hg19/ENCSR064DXG/rep1_1/ENC...
1,ENCSR603TCV,ENCSR064DXG,TOPHAT,hg19,2,HepG2,ENCFF022QUQ.bam,43037edab65acdd2c32584474f58efd9,PTBP1-human,https://www.encodeproject.org/files/ENCFF022QU...,/long-RNA-seq/runs/hg19/ENCSR064DXG/rep2_1/ENC...
2,ENCSR603TCV,ENCSR064DXG,STAR,hg19,1,HepG2,ENCFF145TKU.bam,d8c6fcf3d1ef9650039c150198660498,PTBP1-human,https://www.encodeproject.org/files/ENCFF145TK...,/long-RNA-seq/runs/hg19/ENCSR064DXG/rep1_1/ENC...
3,ENCSR603TCV,ENCSR064DXG,STAR,hg19,2,HepG2,ENCFF855TDM.bam,fd80b7c71a294d57f97396000fd8a762,PTBP1-human,https://www.encodeproject.org/files/ENCFF855TD...,/long-RNA-seq/runs/hg19/ENCSR064DXG/rep2_1/ENC...
4,ENCSR603TCV,ENCSR064DXG,TOPHAT,GRCh38,1,HepG2,ENCFF246XSM.bam,d1d691adbf13a15536dfec8ad949fbff,PTBP1-human,https://www.encodeproject.org/files/ENCFF246XS...,/long-RNA-seq/runs/GRCh38/ENCSR064DXG/rep1_1/E...
5,ENCSR603TCV,ENCSR064DXG,TOPHAT,GRCh38,2,HepG2,ENCFF709DTV.bam,92a3db1e45c7bd5c74a23a719901c53a,PTBP1-human,https://www.encodeproject.org/files/ENCFF709DT...,/long-RNA-seq/runs/GRCh38/ENCSR064DXG/rep2_1/E...
6,ENCSR603TCV,ENCSR064DXG,STAR,GRCh38,1,HepG2,ENCFF211UDA.bam,8f3daa18fc357f2b411c3957ab1dec11,PTBP1-human,https://www.encodeproject.org/files/ENCFF211UD...,/long-RNA-seq/runs/GRCh38/ENCSR064DXG/rep1_1/E...
7,ENCSR603TCV,ENCSR064DXG,STAR,GRCh38,2,HepG2,ENCFF854PFR.bam,5b16b256ee786bf53538c1bdcc1c3e47,PTBP1-human,https://www.encodeproject.org/files/ENCFF854PF...,/long-RNA-seq/runs/GRCh38/ENCSR064DXG/rep2_1/E...
8,ENCSR305XWT,ENCSR052IYH,STAR,hg19,1,HepG2,ENCFF181LSI.bam,60a27c07d4e68382d32e69f841250a46,HNRNPC-human,https://www.encodeproject.org/files/ENCFF181LS...,/long-RNA-seq/runs/hg19/ENCSR052IYH/rep1_1/ENC...
9,ENCSR305XWT,ENCSR052IYH,STAR,hg19,2,HepG2,ENCFF497PPO.bam,073e368d5d67de166971cd8aa92ed8e7,HNRNPC-human,https://www.encodeproject.org/files/ENCFF497PP...,/long-RNA-seq/runs/hg19/ENCSR052IYH/rep2_1/ENC...


In [8]:
samples = list()
error = list()
for h in controls:
    url = experiments+h+"/?format=json"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    if 'code' in data.keys():
        next
    else:
        for i in range(0,len(data['files'])):
            # for key, value in data['files'][i].iteritems():
            #     print(key)
            # print(data['files'][i].iteritems())
            # print( data['files'][i]['replicate']['biological_replicate_number'])
            if ((host+data['files'][i]['href']).endswith('bam') & 
            (data['files'][i]['output_type'] == u'alignments') &
            (data['files'][i]['lab'][u'name'] == 'encode-processing-pipeline')):
                """if(list(X[X['RNASEQ_ControlENC']==h]['RNASEQ_ENCODEAccID'])[0] != data['possible_controls'][0]['accession']):
                    error.append("WARNING: CTRL VS AFF CHANGED: {}:{}:{}".format(
                        h,
                        list(X[X['RNASEQ_ControlENC']==h]['RNASEQ_ENCODEAccID'])[0],
                        data['possible_controls'][0]['accession']))"""
                samples.append({'ctrl_name':data['files'][i]['replicate']['experiment']['target']['name'],
                                'ctrl_cell_line':data['files'][i]['replicate'][u'library']['biosample']['biosample_term_name'],
                                'biological_replicate_number':data['files'][i]['replicate']['biological_replicate_number'],
                                'submitted_ctrl_file_name':data['files'][i]['submitted_file_name'],
                                'ctrl_file_name':os.path.basename(data['files'][i]['href']),
                                'assembly':data['files'][i]['assembly'],
                                'ctrl_url':host+data['files'][i]['href'],
                                'aligner': star_or_tophat(data['files'][i]['submitted_file_name']),
                                'RNASEQ_ControlENC':h,
                                'ctrl_md5sum': data['files'][i]['md5sum']})
                links.append(host+data['files'][i]['href'])
                # controls[data['accession']] = data['possible_controls'][0]['accession']
samples_df = pandas.DataFrame(samples)
samples_df.to_csv(control_manifest,
                  sep='\t',
                  index=None)
samples_df

,RNASEQ_ControlENC,aligner,assembly,biological_replicate_number,ctrl_cell_line,ctrl_file_name,ctrl_md5sum,ctrl_name,ctrl_url,submitted_ctrl_file_name
0,ENCSR603TCV,TOPHAT,hg19,1,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
1,ENCSR603TCV,TOPHAT,hg19,2,HepG2,ENCFF706FVN.bam,b631701769bee3fa3f88e2314ea2a520,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF706FV...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep2_1/ENC...
2,ENCSR603TCV,STAR,hg19,1,HepG2,ENCFF261RLR.bam,40a54cd791c917d4628d19c7bc67efcb,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF261RL...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
3,ENCSR603TCV,STAR,hg19,2,HepG2,ENCFF834TYC.bam,aa67413d0240c95426cafbc30bdd0bd5,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF834TY...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep2_1/ENC...
4,ENCSR603TCV,TOPHAT,GRCh38,1,HepG2,ENCFF892PGW.bam,92c69b912b533ee09228b4cbca0007ea,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF892PG...,/long-RNA-seq/runs/GRCh38/ENCSR603TCV/rep1_1/E...
5,ENCSR603TCV,TOPHAT,GRCh38,2,HepG2,ENCFF733VIR.bam,0bfd56d0b4bad1dbbc182413cb78ae8c,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF733VI...,/long-RNA-seq/runs/GRCh38/ENCSR603TCV/rep2_1/E...
6,ENCSR603TCV,STAR,GRCh38,1,HepG2,ENCFF694JWV.bam,28d37dcd907af4b2c312763941a8e6fb,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF694JW...,/long-RNA-seq/runs/GRCh38/ENCSR603TCV/rep1_1/E...
7,ENCSR603TCV,STAR,GRCh38,2,HepG2,ENCFF902YHV.bam,4a2b59a294dc1b14d3197715f96073a0,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF902YH...,/long-RNA-seq/runs/GRCh38/ENCSR603TCV/rep2_1/E...
8,ENCSR305XWT,STAR,hg19,1,HepG2,ENCFF784SML.bam,75b47d8cf2310c68d9848f51d5a06390,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF784SM...,/long-RNA-seq/runs/hg19/ENCSR305XWT/rep1_1/ENC...
9,ENCSR305XWT,STAR,hg19,2,HepG2,ENCFF487UNM.bam,ad67a8e4b60a4767804fb5866e9ca73e,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF487UN...,/long-RNA-seq/runs/hg19/ENCSR305XWT/rep2_1/ENC...


In [9]:
import pandas as pd # because i like using pd
import numpy as np

In [10]:
samples_df = pd.read_table(sample_manifest)
controls_df = pd.read_table(control_manifest)

In [11]:
samples_df[(samples_df['RNASEQ_ControlENC']=='ENCSR603TCV') & \
           (samples_df['aligner']=='TOPHAT') & \
           (samples_df['assembly']=='hg19') & \
          (samples_df['biological_replicate_number']==1)]

,RNASEQ_ControlENC,RNASEQ_ENCODEAccID,aligner,assembly,biological_replicate_number,expt_cell_line,expt_file_name,expt_md5sum,expt_name,expt_url,submitted_expt_file_name
0,ENCSR603TCV,ENCSR064DXG,TOPHAT,hg19,1,HepG2,ENCFF019ZBE.bam,5a40522cdaea3cef922ca8a2cd2524b4,PTBP1-human,https://www.encodeproject.org/files/ENCFF019ZB...,/long-RNA-seq/runs/hg19/ENCSR064DXG/rep1_1/ENC...
40,ENCSR603TCV,ENCSR017PRS,TOPHAT,hg19,1,HepG2,ENCFF926XIX.bam,92047f19ce9e4554a0102c013e1be7ed,SRSF7-human,https://www.encodeproject.org/files/ENCFF926XI...,/long-RNA-seq/runs/hg19/ENCSR017PRS/rep1_1/ENC...
72,ENCSR603TCV,ENCSR057GCF,TOPHAT,hg19,1,HepG2,ENCFF433FVR.bam,7f77fd5821234c0d05047c35eecf4f3d,TIA1-human,https://www.encodeproject.org/files/ENCFF433FV...,/long-RNA-seq/runs/hg19/ENCSR057GCF/rep1_1/ENC...
80,ENCSR603TCV,ENCSR597XHH,TOPHAT,hg19,1,HepG2,ENCFF665FGJ.bam,e2dfbb1f07f003758f17e72b5dde46b5,SRSF9-human,https://www.encodeproject.org/files/ENCFF665FG...,/long-RNA-seq/runs/hg19/ENCSR597XHH/rep1_1/ENC...
160,ENCSR603TCV,ENCSR148MQK,TOPHAT,hg19,1,HepG2,ENCFF965CYH.bam,dd7dc8358744862998c51c6a746886b4,SF3B4-human,https://www.encodeproject.org/files/ENCFF965CY...,/long-RNA-seq/runs/hg19/ENCSR148MQK/rep1_1/ENC...
488,ENCSR603TCV,ENCSR094KBY,TOPHAT,hg19,1,HepG2,ENCFF448HND.bam,dceed269bc802c0f8760c13587dddb16,SRSF1-human,https://www.encodeproject.org/files/ENCFF448HN...,/long-RNA-seq/runs/hg19/ENCSR094KBY/rep1_1/ENC...
816,ENCSR603TCV,ENCSR998RZI,TOPHAT,hg19,1,HepG2,ENCFF813TQJ.bam,507a30e36a705e27fc08036e05ffc63b,TAF15-human,https://www.encodeproject.org/files/ENCFF813TQ...,/long-RNA-seq/runs/hg19/ENCSR998RZI/rep1_1/ENC...
992,ENCSR603TCV,ENCSR330YOU,TOPHAT,hg19,1,HepG2,ENCFF887BZL.bam,906aa94bb1e6ac5260fc661204a53996,QKI-human,https://www.encodeproject.org/files/ENCFF887BZ...,/long-RNA-seq/runs/hg19/ENCSR330YOU/rep1_1/ENC...
1072,ENCSR603TCV,ENCSR939ZRA,TOPHAT,hg19,1,HepG2,ENCFF685RQP.bam,27d2313707309bc0947d2c6b085b49c6,NELFE-human,https://www.encodeproject.org/files/ENCFF685RQ...,/long-RNA-seq/runs/hg19/ENCSR939ZRA/rep1_1/ENC...
2240,ENCSR603TCV,ENCSR610AEI,TOPHAT,hg19,1,HepG2,ENCFF255BZJ.bam,e9668a86e7faa8acab9ae80521ab1889,RBM25-human,https://www.encodeproject.org/files/ENCFF255BZ...,/long-RNA-seq/runs/hg19/ENCSR610AEI/rep1_1/ENC...


In [12]:
controls_df.drop_duplicates(inplace=True)
controls_df[(controls_df['RNASEQ_ControlENC']=='ENCSR603TCV') & \
           (controls_df['aligner']=='TOPHAT') & \
           (controls_df['assembly']=='hg19') & \
          (controls_df['biological_replicate_number']==1)]

,RNASEQ_ControlENC,aligner,assembly,biological_replicate_number,ctrl_cell_line,ctrl_file_name,ctrl_md5sum,ctrl_name,ctrl_url,submitted_ctrl_file_name
0,ENCSR603TCV,TOPHAT,hg19,1,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...


### perform a pandas merge on the ControlENC, aligner, assembly, and biological replicate number

In [13]:
merged_df = pd.merge(samples_df,controls_df,how='inner',on=['RNASEQ_ControlENC',
                                                            'aligner',
                                                            'assembly',
                                                            'biological_replicate_number'])
merged_df.drop_duplicates(inplace=True)
merged_df

,RNASEQ_ControlENC,RNASEQ_ENCODEAccID,aligner,assembly,biological_replicate_number,expt_cell_line,expt_file_name,expt_md5sum,expt_name,expt_url,submitted_expt_file_name,ctrl_cell_line,ctrl_file_name,ctrl_md5sum,ctrl_name,ctrl_url,submitted_ctrl_file_name
0,ENCSR603TCV,ENCSR064DXG,TOPHAT,hg19,1,HepG2,ENCFF019ZBE.bam,5a40522cdaea3cef922ca8a2cd2524b4,PTBP1-human,https://www.encodeproject.org/files/ENCFF019ZB...,/long-RNA-seq/runs/hg19/ENCSR064DXG/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
1,ENCSR603TCV,ENCSR017PRS,TOPHAT,hg19,1,HepG2,ENCFF926XIX.bam,92047f19ce9e4554a0102c013e1be7ed,SRSF7-human,https://www.encodeproject.org/files/ENCFF926XI...,/long-RNA-seq/runs/hg19/ENCSR017PRS/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
2,ENCSR603TCV,ENCSR057GCF,TOPHAT,hg19,1,HepG2,ENCFF433FVR.bam,7f77fd5821234c0d05047c35eecf4f3d,TIA1-human,https://www.encodeproject.org/files/ENCFF433FV...,/long-RNA-seq/runs/hg19/ENCSR057GCF/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
3,ENCSR603TCV,ENCSR597XHH,TOPHAT,hg19,1,HepG2,ENCFF665FGJ.bam,e2dfbb1f07f003758f17e72b5dde46b5,SRSF9-human,https://www.encodeproject.org/files/ENCFF665FG...,/long-RNA-seq/runs/hg19/ENCSR597XHH/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
4,ENCSR603TCV,ENCSR148MQK,TOPHAT,hg19,1,HepG2,ENCFF965CYH.bam,dd7dc8358744862998c51c6a746886b4,SF3B4-human,https://www.encodeproject.org/files/ENCFF965CY...,/long-RNA-seq/runs/hg19/ENCSR148MQK/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
5,ENCSR603TCV,ENCSR094KBY,TOPHAT,hg19,1,HepG2,ENCFF448HND.bam,dceed269bc802c0f8760c13587dddb16,SRSF1-human,https://www.encodeproject.org/files/ENCFF448HN...,/long-RNA-seq/runs/hg19/ENCSR094KBY/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
6,ENCSR603TCV,ENCSR998RZI,TOPHAT,hg19,1,HepG2,ENCFF813TQJ.bam,507a30e36a705e27fc08036e05ffc63b,TAF15-human,https://www.encodeproject.org/files/ENCFF813TQ...,/long-RNA-seq/runs/hg19/ENCSR998RZI/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
7,ENCSR603TCV,ENCSR330YOU,TOPHAT,hg19,1,HepG2,ENCFF887BZL.bam,906aa94bb1e6ac5260fc661204a53996,QKI-human,https://www.encodeproject.org/files/ENCFF887BZ...,/long-RNA-seq/runs/hg19/ENCSR330YOU/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
8,ENCSR603TCV,ENCSR939ZRA,TOPHAT,hg19,1,HepG2,ENCFF685RQP.bam,27d2313707309bc0947d2c6b085b49c6,NELFE-human,https://www.encodeproject.org/files/ENCFF685RQ...,/long-RNA-seq/runs/hg19/ENCSR939ZRA/rep1_1/ENC...,HepG2,ENCFF674CZO.bam,14cabc9ac6bcceb397aa63bc66228248,Non-specific target control-human,https://www.encodeproject.org/files/ENCFF674CZ...,/long-RNA-seq/runs/hg19/ENCSR603TCV/rep1_1/ENC...
9,ENCSR603TCV,ENCSR610AEI,TOPHAT,hg19,1,HepG2,ENCFF255BZJ.bam,e9668a86e7faa8acab9ae80521ab1889,RBM25-human,https://www.encodeproject.org/files/ENCFF255BZ...,/long-RNA-seq/runs/hg19/ENC

In [14]:
merged_df.to_csv(combined_sample_and_control_manifest,index=None,sep="\t")

### Skip this step if you don't want to re-generate the download script.

In [ ]:
to_download_file = open(to_download,'w')
to_download_file2 = open(to_download2,'w')
for link in list(set(links)):
    to_download_file.write("wget -O {}{} {};\n".format(out_dir, os.path.basename(link), link))
    to_download_file2.write("wget -O {}{} {};\n".format(out_dir2, os.path.basename(link), link))
to_download_file.close()
to_download_file2.close()
len(list(set(links)))

In [ ]:
# ! sh /home/bay001/projects/encode/permanent_data/to_download.sh

In [ ]:
len(links)

In [ ]:
! tail /home/bay001/projects/encode/permanent_data/to_download.sh


In [ ]:
# verify that everything on the manifest looks good:
df = pd.read_table(combined_sample_and_control_manifest)
_test = df[df['expt_name']=='RBFOX2-human']
_testhg19 = _test[_test['assembly']=='hg19']
_testhg19

# Let's check out the file downloaded from ENCODEPROJECT.org

In [57]:
Y = pd.read_table('/home/bay001/projects/encode/permanent_data/metadata.tsv')
Y = Y[(Y['Output type'] == 'alignments') &
      (Y['Lab'] == 'ENCODE Processing Pipeline')]
Y.shape

(3816, 48)

In [61]:
Y.ix[2]

File accession                                                                    ENCFF946IIV
File format                                                                               bam
Output type                                                                        alignments
Experiment accession                                                              ENCSR386YEV
Assay                                                     shRNA knockdown followed by RNA-seq
Biosample term id                                                                 EFO:0002067
Biosample term name                                                                      K562
Biosample type                                                         immortalized cell line
Biosample life stage                                                                    adult
Biosample sex                                                                          female
Biosample organism                                          

In [63]:
X[X['RNASEQ_ENCODEAccID']=='ENCFF946IIV']

,uID,RBP_gID,CellLine,CLIP_rep1,CLIP_rep2,INPUT,RNASEQ_ControlRep1Bam,RNASEQ_ControlRep2Bam,RNASEQ_ControlENC,RNASEQ_KDRep1Bam,RNASEQ_KDRep2Bam,RNASEQ_ENCODEAccID,selection_method,RBP_ENSG
